# Advanced document indexing

# Splitting and ingesting HTML content

## Splitting and ingesting the content of a single URL (on Cornwall)

### Preparing the Chroma DB collections

In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import getpass

OPENAI_API_KEY = getpass.getpass('Enter your OPENAI_API_KEY')

Enter your OPENAI_API_KEY ········


In [2]:
corwnall_granular_collection = Chroma( #A
    collection_name="cornwall_granular",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

corwnall_granular_collection.reset_collection() #B
# A Create a Chorma DB collection
# B Reset the collection in case it already exists 

In [3]:
corwnall_coarse_collection = Chroma( #A 
    collection_name="cornwall_coarse",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

corwnall_coarse_collection.reset_collection() #B
# A Create a Chorma DB collection
# B Reset the collection in case it already exists 

### Loading the HTML content with the AsyncHtmlLoader 

In [4]:
from langchain_community.document_loaders import AsyncHtmlLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
destination_url = "https://en.wikivoyage.org/wiki/Cornwall"

In [6]:
html_loader = AsyncHtmlLoader(destination_url)

In [7]:
docs = html_loader.load()

Fetching pages: 100%|####################################################################| 1/1 [00:01<00:00,  1.00s/it]


In [8]:
len(docs)

1

### Splitting into granular chunks with the HTMLSectionSplitter 

In [20]:
from langchain_text_splitters import HTMLSectionSplitter

In [21]:
headers_to_split_on = [("h1", "Header 1"), ("h2", "Header 2")]
html_section_splitter = HTMLSectionSplitter(
    headers_to_split_on=headers_to_split_on)

In [22]:
def split_docs_into_granular_chunks(docs):
    all_chunks = []
    for doc in docs:
        html_string = doc.page_content #A
        temp_chunks = html_section_splitter.split_text(
            html_string) #B
        all_chunks.extend(temp_chunks) 

    return all_chunks

#A Extract the HTML text from the document
#B Each chunk is a H1 or H2 HTML section

In [12]:
granular_chunks = split_docs_into_granular_chunks(docs)

#### Ingesting granular chunks

In [13]:
corwnall_granular_collection.add_documents(documents=granular_chunks)

['08a7bba3-aef8-476a-ad4e-63c119084db8',
 'a6df53fd-2c1a-484b-9764-9e8c24d94878',
 'c19842a1-25a0-4c73-a745-59ac679a0cde',
 '3d5ae880-7748-466f-a432-7793d797c722',
 '0efbd8a6-a53e-48b5-9f87-4620952b7709',
 '90d8b7d8-9176-4cc8-9ae9-d449b43589ea',
 '458b9960-440a-4a7d-99cf-edba10873f2d',
 '26260235-2662-4f5a-b516-5b2730120f7e',
 'f6299820-35d0-488a-9460-5808c52106f9',
 'd38aa00c-3909-45f6-942a-7ba0ea9c9e4d',
 'e1437b76-4f2c-4e3d-b0e4-ed90266effeb',
 '72dc3656-25d9-46f9-abf8-25b609e16dd0',
 'fcc01cee-8544-4ce0-a28b-99597f0ca0bb',
 '20264649-6b33-43fb-9b3d-985bc4895f90',
 'b32f283b-0518-469d-a324-76a115997bd7',
 '1e0c76a2-52bb-4eeb-af2b-f405bce16127',
 '06dc199e-da80-42e1-8495-0b44e586c4fb',
 '4d627adf-9a4a-470c-b108-19f8e4926dba',
 'da86343c-fad3-4688-a72f-ed05d46d26a0']

#### Searching granular chunks

In [14]:
results = corwnall_granular_collection.similarity_search(
    query="Events or festivals in Cornwall",k=3)
for doc in results:
    print(doc)

page_content='Cornwall' metadata={'Header 1': 'Cornwall'}
page_content='Festivals 
 [ edit ] 
 
 These festivals tend to not be public holidays and not all are celebrated fully across the county. 
   
 AberFest 
 .   A Celtic cultural festival celebrating “All things” Cornish and Breton that takes place biennially (every two years) in Cornwall at Easter. The AberFest Festival alternates with the Breizh – Kernow Festival that is held in Brandivy and Bignan (in Breizh/Bretagne – France) on the alternate years.       ( updated Jun 2023 ) 
 Golowan , sometimes also  Goluan  or  Gol-Jowan  is the Cornish word for the Midsummer celebrations, most popular in the Penwith area and in particular  Penzance  and  Newlyn . The celebrations are conducted from the 23rd of June (St John's Eve) to the 28th of June (St Peter's Eve) each year, St Peter's Eve being the more popular in Cornish fishing communities. The celebrations are centred around the lighting of bonfires and fireworks and the performanc

### Splitting into coarse chunks with the RecursiveCharacterTextSplitter 

In [26]:
from langchain_community.document_transformers import Html2TextTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [27]:
html2text_transformer = Html2TextTransformer()

In [28]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000, chunk_overlap=300
)

In [29]:
def split_docs_into_coarse_chunks(docs):
    text_docs = html2text_transformer.transform_documents(
        docs) #A 
    coarse_chunks = text_splitter.split_documents(
        text_docs)

    return coarse_chunks
#A transform HTML docs into clean text docs

In [30]:
coarse_chunks = split_docs_into_coarse_chunks(docs)

#### Ingesting coarse chunks

In [31]:
corwnall_coarse_collection.add_documents(documents=coarse_chunks)

['9b1bd257-ac58-4587-8ff1-ecfcc6a8a7cf',
 '907be2db-d885-45fd-9518-2981cd8def0f',
 '3947fff4-adcd-4ee4-9902-e1464c41d978',
 'b4fa60b4-70f6-40b8-b4b8-cd7214f58e8e',
 'e45e800f-2748-4b18-aa85-c7cc275f5ea9',
 '95e8d917-9ac5-482e-853c-c37c1e0346c6',
 '1ba4c161-711d-49b1-953c-343b112c1d5d',
 'c703e146-b3b2-4728-8388-14d49bc3d352',
 '1cf6e1f4-dcdf-4266-9ea2-80486ab5207a',
 'e655bc6c-51d3-46a5-bffc-a499cc69a920',
 '71187ed0-20a6-4404-a7b2-917309042949',
 'dce08f10-50d1-4681-8f56-77a5a6c7d546',
 'b4419069-6ea1-4fe5-9d75-852816999dd8',
 '0959a45a-20e3-4e37-9ff6-e9a6b1e8ec61',
 '2893d8ff-b5f7-4f05-99d7-2bc9c2433539']

#### Searching coarse chunks

In [32]:
results = corwnall_coarse_collection.similarity_search(
    query="Events or festivals in Cornwall",k=3)
for doc in results:
    print(doc)

page_content='### Spirits

[edit]

    _See also:Liquor_

Gin and rum are also produced in Cornwall. A popular brand of Cornish rum is
Dead Man's Fingers which has multiple flavours and is bottled in St. Ives.

## Festivals

[edit]

These festivals tend to not be public holidays and not all are celebrated
fully across the county.

AberFest. A Celtic cultural festival celebrating “All things” Cornish and
Breton that takes place biennially (every two years) in Cornwall at Easter.
The AberFest Festival alternates with the Breizh – Kernow Festival that is
held in Brandivy and Bignan (in Breizh/Bretagne – France) on the alternate
years. (updated Jun 2023)

**Golowan** , sometimes also _Goluan_ or _Gol-Jowan_ is the Cornish word for
the Midsummer celebrations, most popular in the Penwith area and in particular
Penzance and Newlyn. The celebrations are conducted from the 23rd of June (St
John's Eve) to the 28th of June (St Peter's Eve) each year, St Peter's Eve
being the more popular in Corni

## Splitting and ingesting the content of various URLs (across UK destinations)

### Preparing the Chroma DB collections

In [33]:
uk_granular_collection = Chroma( #A
    collection_name="uk_granular",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

uk_granular_collection.reset_collection() #B

In [34]:
uk_coarse_collection = Chroma( #A
    collection_name="uk_coarse",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

uk_coarse_collection.reset_collection() #B

### Splitting and ingesting HTML content with the HTMLSectionSplitter 

In [35]:
# Reduce this list if you want to save on processing fees
uk_destinations = [
    "Cornwall", "North_Cornwall", "South_Cornwall", "West_Cornwall", 
    "Tintagel", "Bodmin", "Wadebridge", "Penzance", "Newquay",
    "St_Ives", "Port_Isaac", "Looe", "Polperro", "Porthleven"
    "East_Sussex", "Brighton", "Battle", "Hastings_(England)", 
    "Rye_(England)", "Seaford", "Ashdown_Forest"
] 

wikivoyage_root_url = "https://en.wikivoyage.org/wiki"

In [36]:
uk_destination_urls = [f'{wikivoyage_root_url}/{d}' for d in uk_destinations]

In [37]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #C
    docs =  html_loader.load() #D
    
    for doc in docs:
        print(doc.metadata)
        granular_chunks = split_docs_into_granular_chunks(docs)
        uk_granular_collection.add_documents(documents=granular_chunks)

        coarse_chunks = split_docs_into_coarse_chunks(docs)
        uk_coarse_collection.add_documents(documents=coarse_chunks)
#A Create a Chroma DB collection
#B Reset the collection in case it already exists 
#C Loader for one destination
#D Documents of one destination 

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  7.50it/s]


{'source': 'https://en.wikivoyage.org/wiki/Cornwall', 'title': 'Cornwall – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  1.48it/s]


{'source': 'https://en.wikivoyage.org/wiki/North_Cornwall', 'title': 'North Cornwall – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  1.58it/s]


...

{'source': 'https://en.wikivoyage.org/wiki/Rye_(England)', 'title': 'Rye (England) – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  1.64it/s]


{'source': 'https://en.wikivoyage.org/wiki/Seaford', 'title': 'Seaford – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 14.67it/s]


{'source': 'https://en.wikivoyage.org/wiki/Ashdown_Forest', 'title': 'Ashdown Forest – Travel guide at Wikivoyage', 'language': 'en'}


#### Searching 

In [38]:
granular_results = uk_granular_collection.similarity_search(
    query="Events or festivals in East Sussex",k=4)
for doc in granular_results:
    print(doc)

page_content='Brighton' metadata={'Header 1': 'Brighton'}
page_content='South Cornwall' metadata={'Header 1': 'South Cornwall'}
page_content='Seaford' metadata={'Header 1': 'Seaford'}
page_content='Penzance' metadata={'Header 1': 'Penzance'}


In [39]:
coarse_results = uk_coarse_collection.similarity_search(
    query="Events or festivals in East Sussex",k=4)
for doc in coarse_results:
    print(doc)

page_content='### Events

[edit]

A market during the Brighton Festival. The Theatre Royal is the red building.
A colourful parade down Queens Road during Pride in 2016.

  * **Brighton Racecourse** has flat-racing April-Oct. It's on Freshfield Rd a mile east of town centre.
  * **Plumpton Racecourse** is National Hunt (jumps races) Nov-March, but it's 10 mi (16 km) north in Lewes.
  * Brighton Festival Fringe: early May – early June, ☏ +44 1273 764900, info@brightonfringe.org. The Fringe runs at the same time as the main festival, and features over 600 events, including comedy, theatre, music, and "open houses" (local artists exhibiting in their own homes) and tours (haunted pubs, Regency Brighton, churches, cemeteries, sewers, etc.)_ (date needs fixing)_
  * Brighton Festival: May, ☏ +44 1273 709709 (tickets), tickets@brightonfestival.org. The Brighton Festival, in May each year, is the second biggest arts festival in Great Britain (coming closely behind Edinburgh). Music of all sort

In [40]:
granular_results = uk_granular_collection.similarity_search(
    query="Beaches in Conrwall",k=4)
for doc in granular_results:
    print(doc)

page_content='Cornwall' metadata={'Header 1': 'Cornwall'}
page_content='North Cornwall' metadata={'Header 1': 'North Cornwall'}
page_content='West Cornwall' metadata={'Header 1': 'West Cornwall'}
page_content='South Cornwall' metadata={'Header 1': 'South Cornwall'}


In [41]:
coarse_results = uk_coarse_collection.similarity_search(
    query="Beaches in Cornwall",k=4)
for doc in coarse_results:
    print(doc)

page_content='**South Cornwall** is in Cornwall. It includes much of the stunning Cornish
coast along the English Channel of the Atlantic Ocean.

## Towns and villages

[edit]

Map of South Cornwall

  * 50.26-5.0511 Truro — Cornwall's main centre hosts the Royal Cornwall Museum
  * 50.3311-4.20212 Cawsand — overlooks Plymouth Sound; Cawsand is within Mount Edgcumbe Country Park
  * 50.15-5.073 Falmouth — famous for its beaches, it is home to the world's third largest natural harbour
  * 50.334-4.6334 Fowey — the Fowey Regatta in mid-August attracts many yachts and sailing boats
  * 50.354-4.4545 Looe — a summer resort place with a monkey sanctuary, and an active fishing village
  * 50.408-4.2126 Saltash — "Gateway to Cornwall", a small town on the Cornwall side of the Tamar crossings
  * 50.338-4.7957 St Austell — largest town in the county and home to the Eden Project, the world's largest greenhouse
  * 50.3314-4.75788 Charlestown — seaside town used as filming location for the TV sh

# Embedding strategy

## Embedding child chunks with ParentDocumentRetriever

In [42]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

### Setting up the Parent Document retriever

In [43]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=3000) #A
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500) #B

child_chunks_collection = Chroma( #C
    collection_name="uk_child_chunks",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

child_chunks_collection.reset_collection() #D

doc_store = InMemoryStore() #E

parent_doc_retriever = ParentDocumentRetriever( #F
    vectorstore=child_chunks_collection,
    docstore=doc_store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)
#A Splitter to generate parent coarse chunks from original documents (parsed from web pages)
#B Splitter to generate child granular chunks from parent coarse chunks
#C Vector store collection to host child granular chunks
#D Make sure the collection is empty
#E Document store to host parent coarse chunks
#F Retriever to link parent coarse chunks to child granular chunks

### Ingesting the content into doc and vector store

In [44]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(html_docs) #C

    print(f'Ingesting {destination_url}')
    parent_doc_retriever.add_documents(text_docs, ids=None) #D

#A Loader for destination web page
#B HTML documents of one destination 
#C Transform HTML docs into clean text deocs
#D Ingest coarse chunks into document store and granular chunks into vector store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 12.12it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 18.28it/s]


Ingesting https://en.wikivoyage.org/wiki/North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 19.87it/s]


...

Ingesting https://en.wikivoyage.org/wiki/Rye_(England)


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 21.35it/s]


Ingesting https://en.wikivoyage.org/wiki/Seaford


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 18.53it/s]


Ingesting https://en.wikivoyage.org/wiki/Ashdown_Forest


In [45]:
list(doc_store.yield_keys())
#A Show the keys of the added coarse chunks

['86f63110-59a0-4344-b582-1096dd74eb63',
 'd8c97c44-26ae-48ad-9aee-3f8f05f4223e',
 '6b530844-8984-47b5-ad18-27e1dd614f61',
 '4c63bf97-7abc-4833-bc85-3f66a53d46c0',
 'e02ccaa3-c6af-460a-93b5-de5866e251d2',
 '24c3ac24-3904-455c-adf2-7d40696f079c',
 'dcbcd23b-209f-43e1-898c-dbad8b6566c7',
 'ad0a61de-9fdd-4b9a-ae94-5e25183d56d1',
 'b0de955b-add4-44b4-84d4-5f48e0d4f8de',
 'd81f11c6-7f10-42e7-954b-ac96ffe738ef',
 '5f6f322f-4013-417c-ae4d-c278885c4dcf',
 'ff3106c3-5ea3-4859-9a0d-4b2c77cccd79',
 '57ac754e-4018-45df-94e9-4d23359a484c',
 '1c66a20c-6814-4454-8ee7-e421bc153747',
 '572753bc-e80e-4410-85cc-6ab9d7019e0f',
 '4f73a9d3-abad-4e4f-abc2-44a27f32ebad',
 '14ca2a71-1adc-4073-964f-8b4b54c5ee57',
 '78baa414-89e4-4da0-a1ac-c7ccfbdc18f1',
 '7bc19ba2-da18-47a3-9fbc-1978a2308fd2',
 '60f6db9a-20c1-43bd-a42a-4466d0acc92b',
 'b4c79572-c27f-4063-a3e6-5fefc79dd0e0',
 'b2cc3a16-3769-4213-9532-68126c695697',
 '304afdfd-b437-42d2-95d8-27c1850afe6b',
 '740e812c-7dfd-4a5f-adda-0177ed68fbd3',
 'a05cf1da-374e-

### Performing a search on granular information 

In [46]:
retrieved_docs = parent_doc_retriever.invoke("Cornwall Ranger")

In [47]:
len(retrieved_docs)

4

In [48]:
retrieved_docs[0]

Document(metadata={'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage', 'language': 'en'}, page_content="## Get around\n\n[edit]\n\n### By bus\n\n[edit]\n\nThanks to Transport for Cornwall, all bus tickets are interchangeable across\nthe different companies. The **Cornwall All Day ticket** allows unlimited\ntravel for a calendar day. As of 2023, fares are £5 for adults and £4 for\nunder-19s. Payment is by cash or contactless. The two main bus companies are:\n\n  * **Go Cornwall Bus** covers all parts of Cornwall and connects with Plymouth (in Devon).\n  * **Kernow** (part of First Bus) covers western and central Cornwall.\n\nBuses only serve designated stops when in towns; otherwise, you can flag them\ndown anywhere that's safe for them to stop.\n\n### By train\n\n[edit]\n\n**CrossCountry Trains** and **Great Western Railway** operate regular train\nservices between the main centres of population, the latter company also\nse

### Comparing with direct semantic search on child chunks

In [49]:
child_docs_only =  child_chunks_collection.similarity_search("Cornwall Ranger")

In [50]:
len(child_docs_only)

4

In [51]:
child_docs_only[0]

Document(id='00661766-7365-4d88-8607-819860932619', metadata={'doc_id': '304afdfd-b437-42d2-95d8-27c1850afe6b', 'language': 'en', 'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage'}, page_content='The **Cornwall Ranger** ticket allows unlimited train travel in Cornwall and\nPlymouth for a calendar day. As of 2023, this costs £14 for adults and £7 for\nunder-16s.\n\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.')

In [51]:
# IMPORTANT: as you can see a granular search would have identified the chunk, but it would have lost the usefulcontext about travelling in Cornwall

## Embedding child chunks with MultiVectorRetriever

In [60]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import uuid

### Setting up the Multi vector retriever

In [61]:
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000) #A
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500) #B

child_chunks_collection = Chroma( #C
    collection_name="uk_child_chunks",
    embedding_function=OpenAIEmbeddings(
        openai_api_key=OPENAI_API_KEY),
)

child_chunks_collection.reset_collection() #D

doc_byte_store = InMemoryByteStore() #E
doc_key = "doc_id"

multi_vector_retriever = MultiVectorRetriever( #F
    vectorstore=child_chunks_collection,
    byte_store=doc_byte_store
)
#A Splitter to generate parent coarse chunks from original documents (parsed from web pages)
#B Splitter to generate child granular chunks from parent coarse chunks
#C Vector store collection to host child granular chunks
#D Make sure the collection is empty
#E Document store to host parent coarse chunks
#F Retriever to link parent coarse chunks to child granular chunks

### Ingesting the content into doc and vector store

In [62]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(
        html_docs) #C

    coarse_chunks = parent_splitter.split_documents(
        text_docs) #D

    coarse_chunks_ids = [str(uuid.uuid4()) for _ in coarse_chunks]
    all_granular_chunks = []
    for i, coarse_chunk in enumerate(
        coarse_chunks): #E
        
        coarse_chunk_id = coarse_chunks_ids[i]
            
        granular_chunks = child_splitter.split_documents(
            [coarse_chunk]) #F

        for granular_chunk in granular_chunks:
            granular_chunk.metadata[doc_key] = coarse_chunk_id #G

        all_granular_chunks.extend(granular_chunks)

    print(f'Ingesting {destination_url}')
    multi_vector_retriever.vectorstore.add_documents(
        all_granular_chunks) #H
    multi_vector_retriever.docstore.mset(
        list(zip(coarse_chunks_ids, coarse_chunks))) #I

#A Loader for one destination
#B Documents of one destination 
#C transform HTML docs into clean text docs
#D Split the destination content into parent coarse chunks
#E Iterate over the parent coarse chunks
#F Create child granular chunks form each parent coarse chunk
#G Link each child granular chunk to its parent coarse chunk
#H Ingest the child granular chunks into the vector store
#I Ingest the parent coarse chunks into the document store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 12.50it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 18.71it/s]


Ingesting https://en.wikivoyage.org/wiki/North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 21.26it/s]


...

Ingesting https://en.wikivoyage.org/wiki/Rye_(England)


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 11.15it/s]


Ingesting https://en.wikivoyage.org/wiki/Seaford


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 20.57it/s]


Ingesting https://en.wikivoyage.org/wiki/Ashdown_Forest


### Performing a search on granular information

In [63]:
retrieved_docs = multi_vector_retriever.invoke(
    "Cornwall Ranger")

In [64]:
len(retrieved_docs)

4

In [65]:
retrieved_docs[0]

Document(metadata={'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage', 'language': 'en'}, page_content="## Get around\n\n[edit]\n\n### By bus\n\n[edit]\n\nThanks to Transport for Cornwall, all bus tickets are interchangeable across\nthe different companies. The **Cornwall All Day ticket** allows unlimited\ntravel for a calendar day. As of 2023, fares are £5 for adults and £4 for\nunder-19s. Payment is by cash or contactless. The two main bus companies are:\n\n  * **Go Cornwall Bus** covers all parts of Cornwall and connects with Plymouth (in Devon).\n  * **Kernow** (part of First Bus) covers western and central Cornwall.\n\nBuses only serve designated stops when in towns; otherwise, you can flag them\ndown anywhere that's safe for them to stop.\n\n### By train\n\n[edit]\n\n**CrossCountry Trains** and **Great Western Railway** operate regular train\nservices between the main centres of population, the latter company also\nse

In [36]:
##IMPORTANT: same as Parent Document retriever, but more control and flexibility on how to link child to parent chunks

### Comparing with direct semantic search on child chunks

In [69]:
child_docs_only =  child_chunks_collection.similarity_search(
    "Cornwall Ranger")

In [70]:
len(child_docs_only)

4

In [71]:
child_docs_only[0]

Document(id='a6d93c6d-5ff5-4ca6-a2a2-eafb9b17d44c', metadata={'doc_id': '43d95135-a5dd-4730-85f2-d0b7375e00c8', 'language': 'en', 'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage'}, page_content='The **Cornwall Ranger** ticket allows unlimited train travel in Cornwall and\nPlymouth for a calendar day. As of 2023, this costs £14 for adults and £7 for\nunder-16s.\n\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.')

In [72]:
## IMPORTANT: Same as before

## Embedding summaries with MultiVectorRetriever

In [81]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import uuid

### Setting up the Multi vector retriever (similar to when embedding child chunks)

In [82]:
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000) #A

summaries_collection = Chroma( #B
    collection_name="uk_summaries",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

summaries_collection.reset_collection() #C

doc_byte_store = InMemoryByteStore() #D
doc_key = "doc_id"

multi_vector_retriever = MultiVectorRetriever( #E
    vectorstore=summaries_collection,
    byte_store=doc_byte_store
)
#A Splitter to generate parent coarse chunks from original documents (parsed from web pages)
#B Vector store collection to host child granular chunks
#C Make sure the collection is empty
#D Document store to host parent coarse chunks
#E Retriever to link parent coarse chunks to child granular chunks

### Setting up the summarization chain

In [83]:
llm = ChatOpenAI(model="gpt-5-nano", openai_api_key=OPENAI_API_KEY)

In [84]:
summarization_chain = (
    {"document": lambda x: x.page_content} #A
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{document}") #B
    | llm
    | StrOutputParser())

#A Grab the text content from the document
#B Instantiate a prompt asking to generate summary of the provided text
#C Send the LLM the instantiated prompt 
#D Extract the summary text from the response

### Ingesting the coarse chunks and related summaries into doc and vector store

In [85]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(
        html_docs) #C

    coarse_chunks = parent_splitter.split_documents(
        text_docs) #D

    coarse_chunks_ids = [str(uuid.uuid4()) for _ in coarse_chunks]
    all_summaries = []
    for i, coarse_chunk in enumerate(
        coarse_chunks): #E
        
        coarse_chunk_id = coarse_chunks_ids[i]
            
        summary_text =  summarization_chain.invoke(
            coarse_chunk) #F
        summary_doc = Document(page_content=summary_text, 
                               metadata={doc_key: coarse_chunk_id})

        all_summaries.append(summary_doc) #G

    print(f'Ingesting {destination_url}')
    multi_vector_retriever.vectorstore.add_documents(
        all_summaries) #H
    multi_vector_retriever.docstore.mset(
        list(zip(coarse_chunks_ids, coarse_chunks))) #I

#A Loader for one destination
#B Documents of one destination 
#C transform HTML docs into clean text docs
#D Split the destination content into coarse chunks
#E Iterate over the coarse chunks
#F Generate a summary for the coarse chunk thorugh the summarization chain
#G Link each summary to its related coarse chunk
#H Ingest the summaries into the vector store
#I Ingest the coarse chunks into the document store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 15.61it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 10.52it/s]


Ingesting https://en.wikivoyage.org/wiki/North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 18.03it/s]


...

Ingesting https://en.wikivoyage.org/wiki/Penzance


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 19.66it/s]


Ingesting https://en.wikivoyage.org/wiki/Newquay


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 19.28it/s]


KeyboardInterrupt: 

In [ ]:
# COMMENT: the code above is similar to when ingesting child chunks, but it is slower because of the summarization step
# which invokes the LLM.
# The processing can be speeded up by parallelizing the outer for loop on the destination urls.

### Performing a search on granular information

In [ ]:
retrieved_docs = multi_vector_retriever.invoke("Cornwall travel")

In [ ]:
len(retrieved_docs)

In [73]:
retrieved_docs

[Document(metadata={'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage', 'language': 'en'}, page_content="Trains from London take about 3 hr 20 min to Plymouth.\n\n### By car\n\n[edit]\n\nCornwall can be accessed by road via the A30 which runs from the end of the M5\nat Exeter, all the way through the heart of Devon and Cornwall down to Land's\nEnd. It is a grade-separated expressway as far as Carland Cross near Truro\n(the expressway is expected to be open as far as Camborne (between Redruth and\nHayle) by March 2024). You can also get to Cornwall via the A38, crossing the\nRiver Tamar at Plymouth via the Tamar Bridge, which levies a toll on eastbound\nvehicles. On summer Saturdays and during bank holiday weekends roads to\nCornwall are usually busy.\n\n## Get around\n\n[edit]\n\n### By bus\n\n[edit]\n\nThanks to Transport for Cornwall, all bus tickets are interchangeable across\nthe different companies. The **Cornwall All 

### Comparing with direct semantic search on summaries

In [ ]:
summary_docs_only =  summaries_collection.similarity_search(
    "Cornwall Travel")

In [75]:
len(summary_docs_only)

4

In [76]:
summary_docs_only

[Document(metadata={'doc_id': '68673c9f-4ae9-4026-8f73-abd7218557f6'}, page_content="The document is a travel guide excerpt from Wikivoyage about Cornwall, a county in the southwest of the United Kingdom. It describes Cornwall as a distinct and popular tourist destination known for its warm climate, long coastline, stunning scenery, and rich Celtic heritage, including legends of pirates and King Arthur. The region is recognized for its cultural tourism and archaeological significance, with over 30% of the area designated as an Area of Outstanding Natural Beauty (AONB).\n\nThe guide outlines various sections, including regions, towns and cities, visitor information, transportation options, attractions like National Trust properties, local cuisine, beverages, festivals, accommodation, and safety tips. Cornwall is divided into three main regions: North Cornwall, South Cornwall, and West Cornwall, each offering unique landscapes and experiences. The document emphasizes the area's pride in 

In [77]:
# COMMENT: a direct search on summaries retrieves denser information, but it is missing out on useful details. 
# However, you might consider using the summaries directly if after testing they prove adequate.

## Embedding hypothetical questions with MultiVectorRetriever

In [97]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import uuid
from typing import List
from pydantic import BaseModel, Field

### Setting up the Multi vector retriever (same as when embedding summaries)

In [98]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=3000) #A

hypothetical_questions_collection = Chroma( #B
    collection_name="uk_hypothetical_questions",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

hypothetical_questions_collection.reset_collection() #C

doc_byte_store = InMemoryByteStore() #D
doc_key = "doc_id"

multi_vector_retriever = MultiVectorRetriever( #E
    vectorstore=hypothetical_questions_collection,
    byte_store=doc_byte_store
)
#A Splitter to generate parent coarse chunks from original documents (parsed from web pages)
#B Vector store collection to host child granular chunks
#C Make sure the collection is empty
#D Document store to host parent coarse chunks
#E Retriever to link parent coarse chunks to child granular chunks

### Setting up the chain to generate hypothetical questions

In [99]:
class HypotheticalQuestions(BaseModel):
    """Generate hypothetical questions for given text."""

    questions: List[str] = Field(..., description="List of hypothetical questions for given text")

In [100]:
llm_with_structured_output = ChatOpenAI(
    model="gpt-5-nano", 
    openai_api_key=OPENAI_API_KEY).with_structured_output(
        HypotheticalQuestions
)

In [101]:
hypothetical_questions_chain = (
    {"document_text": lambda x: x.page_content} #A
    | ChatPromptTemplate.from_template( #B
        "Generate a list of exactly 4 hypothetical questions that the below text could be used to answer:\n\n{document_text}"
    )
    | llm_with_structured_output #C
    | (lambda x: x.questions) #D
)

#A Grab the text content from the document
#B Instantiate a prompt asking to generate 4 hypothetical questions on the provided text
#C Invoke the LLM configured to return an object containing the questions as a typed list of strings
#D Grab the list of questions from the response

### Ingesting the coarse chunks and related hypothetical questions into doc and vector store

In [102]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(
        html_docs) #C

    coarse_chunks = parent_splitter.split_documents(
        text_docs) #D

    coarse_chunks_ids = [str(uuid.uuid4()) for _ in coarse_chunks]
    all_hypothetical_questions = []
    for i, coarse_chunk in enumerate(
        coarse_chunks): #E
        
        coarse_chunk_id = coarse_chunks_ids[i]
            
        hypothetical_questions = hypothetical_questions_chain.invoke(
            coarse_chunk) #F
        hypothetical_questions_docs = [Document(
            page_content=question, metadata={doc_key: coarse_chunk_id})
                    for question 
                    in hypothetical_questions] #G

        all_hypothetical_questions.extend(hypothetical_questions_docs)

    print(f'Ingesting {destination_url}')
    multi_vector_retriever.vectorstore.add_documents(
        all_hypothetical_questions) #H
    multi_vector_retriever.docstore.mset(
        list(zip(coarse_chunks_ids, coarse_chunks))) #I

#A Loader for one destination
#B Documents of one destination 
#C transform HTML docs into clean text docs
#D Split the destination content into coarse chunks
#E Iterate over the coarse chunks
#F Generate a list of hypothetical questions for the coarse chunk thorugh the question generation chain
#G Link each hypothetical question to its related coarse chunk
#H Ingest the hypothetical questions into the vector store
#I Ingest the coarse chunks into the document store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 14.63it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 17.21it/s]


KeyboardInterrupt: 

### Performing a search on granular information

In [93]:
retrieved_docs = multi_vector_retriever.invoke(
    "How can you go to Brighton from London?")

In [94]:
len(retrieved_docs)

4

In [95]:
retrieved_docs

[Document(metadata={'source': 'https://en.wikivoyage.org/wiki/Newquay', 'title': 'Newquay – Travel guide at Wikivoyage', 'language': 'en'}, page_content='[edit]\n\nFollow the M5 to Exeter, then take the A30 to Newquay. Leave this road near\nIndian Queens and continue on the A39 and then A392 which takes you directly\ninto the town.\n\n### By train\n\n[edit]\n\n50.415-5.0751 Newquay station is the terminus of a branch line that connects\nto the main line at Par. From Par, trains run hourly to London Paddington (4\nhr 30 min, via Plymouth, Exeter and Reading) and to Penzance (1 hour); other\ndestinations may involve a change at Plymouth or Exeter. There are eight\ntrains a day between Par and Newquay, including daily through intercity trains\nbetween London and Newquay from May to September. If your main line train is\nlate and you\'re going to miss your connecting train to Newquay at Par, speak\nto the conductor who will either hold the connection or organise other\ntransport. You also 

### Inspecting possible questions matching our question through semantic search

In [96]:
hypothetical_question_docs_only = hypothetical_questions_collection.similarity_search(
    "How can you go to Brighton from London?")

NameError: name 'hypothetical_questions_collection' is not defined

In [68]:
len(hypothetical_question_docs_only)

4

In [69]:
hypothetical_question_docs_only

[Document(id='91837b61-f3c7-4fe2-9c81-56e660b68998', metadata={'doc_id': '1c0a73eb-9f2d-414c-b7e7-878d4509f0bd'}, page_content='What if someone wanted to plan a day trip from London to Brighton, what transportation options could they consider?'),
 Document(id='d19d9895-1c16-4ef2-86b5-e231fc0b68f8', metadata={'doc_id': 'f16aea58-1ed5-4c7a-a0ef-676e64cd55e6'}, page_content='What would it be like to travel to Brighton from Gatwick Airport using the train?'),
 Document(id='1bd3353f-2afa-4cba-9790-aec9a1d546fa', metadata={'doc_id': 'd9c56504-d613-447a-8c00-56f5ac39d10d'}, page_content='How could someone navigate around Brighton using public transportation?'),
 Document(id='b6822b8b-3193-48bc-a2f3-f47c7d2e8628', metadata={'doc_id': 'd9c56504-d613-447a-8c00-56f5ac39d10d'}, page_content='What would a traveler need to know before visiting Brighton?')]

# Granular chunk expansion with MultiVectorRetriever

In [70]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import uuid

### Setting up the Multi vector retriever

In [71]:
granular_chunk_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500) #A

granular_chunks_collection = Chroma( #B
    collection_name="uk_granular_chunks",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

granular_chunks_collection.reset_collection() #C

expanded_chunk_store = InMemoryByteStore() #D
doc_key = "doc_id"

multi_vector_retriever = MultiVectorRetriever( #E
    vectorstore=granular_chunks_collection,
    byte_store=expanded_chunk_store
)
#A Splitter to generate granular chunks from original documents (parsed from web pages)
#B Vector store collection to host child granular chunks
#C Make sure the collection is empty
#D Document store to host expanded chunks
#E Retriever to link parent coarse chunks to child granular chunks

### Ingesting granular and expanded chunks into doc and vector store

In [72]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(
        html_docs) #C

    granular_chunks = granular_chunk_splitter.split_documents(
        text_docs) #D

    expanded_chunk_store_items = []
    for i, granular_chunk in enumerate(
        granular_chunks): #E

        this_chunk_num = i #F
        previous_chunk_num = i-1 #F
        next_chunk_num = i+1 #F
        
        if i==0: #F
            previous_chunk_num = None
        elif i==(len(granular_chunks)-1): #F
            next_chunk_num = None

        expanded_chunk_text = "" #G
        if previous_chunk_num: #G
            expanded_chunk_text += granular_chunks[
                previous_chunk_num].page_content
            expanded_chunk_text += "\n"

        expanded_chunk_text += granular_chunks[
            this_chunk_num].page_content #G
        expanded_chunk_text += "\n"

        if next_chunk_num: #G
            expanded_chunk_text += granular_chunks[
                next_chunk_num].page_content
            expanded_chunk_text += "\n"

        expanded_chunk_id = str(uuid.uuid4()) #H
        expanded_chunk_doc = Document(
            page_content=expanded_chunk_text) #I

        expanded_chunk_store_item = (expanded_chunk_id, 
                                     expanded_chunk_doc)
        expanded_chunk_store_items.append(
            expanded_chunk_store_item)

        granular_chunk.metadata[
            doc_key] = expanded_chunk_id #J
            
    print(f'Ingesting {destination_url}')
    multi_vector_retriever.vectorstore.add_documents(
        granular_chunks) #K
    multi_vector_retriever.docstore.mset(
        expanded_chunk_store_items) #L

#A Loader for one destination
#B Documents of one destination 
#C transform HTML docs into clean text docs
#D Split the destination content into granular chunks
#E Iterate over the granular chunks
#F determine the index of the current chunk and its previous and next chunks
#G Assemble the text of the expanded chunk by including the previous and next chunk
#H Generate the ID of the expanded chunk
#I Create the expanded chunk document
#J Link each granular chunk to its related expanded chunk
#K Ingest the granular chunks into the vector store
#L Ingest the expanded chunks into the document store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 14.71it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 17.54it/s]


Ingesting https://en.wikivoyage.org/wiki/North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 13.44it/s]


...

Ingesting https://en.wikivoyage.org/wiki/Rye_(England)


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 18.29it/s]


Ingesting https://en.wikivoyage.org/wiki/Seaford


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 13.52it/s]


Ingesting https://en.wikivoyage.org/wiki/Ashdown_Forest


### Performing a search on granular information

In [193]:
retrieved_docs = multi_vector_retriever.invoke("Cornwall Ranger")

In [194]:
len(retrieved_docs)

4

In [195]:
retrieved_docs[0]

Document(page_content="Buses only serve designated stops when in towns; otherwise, you can flag them\ndown anywhere that's safe for them to stop.\n\n### By train\n\n[edit]\n\n**CrossCountry Trains** and **Great Western Railway** operate regular train\nservices between the main centres of population, the latter company also\nserving a number of other towns on branch lines. For train times and fares\nvisit National Rail Enquiries.\nThe **Cornwall Ranger** ticket allows unlimited train travel in Cornwall and\nPlymouth for a calendar day. As of 2023, this costs £14 for adults and £7 for\nunder-16s.\n\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.\n\nThe **Lost Gardens of

### Comparing with direct semantic search on granular chunks

In [196]:
child_docs_only =  child_chunks_collection.similarity_search("Cornwall Ranger")

In [197]:
len(child_docs_only)

4

In [198]:
child_docs_only[0]

Document(metadata={'doc_id': '04c7f88e-e090-4057-af5b-ea584e777b3f', 'language': 'en', 'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage'}, page_content='The **Cornwall Ranger** ticket allows unlimited train travel in Cornwall and\nPlymouth for a calendar day. As of 2023, this costs £14 for adults and £7 for\nunder-16s.\n\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.')

In [ ]:
# COMMENT: the expanded chunk has more useful context